### Context
A way to pass along data during the agent's lifecycle.

It is NOT available directly to the agent, but rather to the functions/tools it has access to.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [2]:
from dataclasses import dataclass
import random

from agents import Agent, RunContextWrapper, Runner, TResponseInputItem, function_tool


@dataclass
class UserProfile:
    id: str
    name: str
    shopping_cart: list[str]

@function_tool
async def get_budget(wrapper: RunContextWrapper[UserProfile]):
    """
    Get the account balance of the user using the user's id and their linked bank account
    """
    print("Getting account balance")
    user_id = wrapper.context.id

    # pretend we are fetching the account balance from a database

    return 100.0

@function_tool
async def search_for_item(wrapper: RunContextWrapper[UserProfile], item: str) -> str:
    """
    Search for an item in the database
    """
    print("Searching for item")
    # randomly generate a price for the item
    price = random.randint(1, 100)
    return f"Found {item} in the database for ${price}.00"

@function_tool
async def get_shopping_cart(wrapper: RunContextWrapper[UserProfile]) -> list[str]:
    print("Getting shopping cart")
    return wrapper.context.shopping_cart

@function_tool
async def add_to_shopping_cart(wrapper: RunContextWrapper[UserProfile], items: list[str]) -> None:
    print("Adding items to shopping cart")
    wrapper.context.shopping_cart.extend(items)
    
@function_tool
async def purchase_items(wrapper: RunContextWrapper[UserProfile]) -> None:
    print("Purchasing items")
    
    # we could take the items from the shopping cart and purchase them using some external API
    # for now, we'll just print a message
    print(f"Successfully purchased items: {wrapper.context.shopping_cart}")

shopping_agent = Agent[UserProfile](
    name="Shopping Assistant",
    instructions=(
        "You are a shopping assistant dedicated to helping the user with their grocery shopping needs."
        "Your primary role is to assist in creating a shopping plan that fits within the user's budget."
        "Start by getting the user's budget using the tool get_budget."
        "Provide suggestions for items if requested, and always aim to keep the total cost within the user's budget."
        "If the user is nearing or exceeding their budget, inform them and suggest alternatives or adjustments to the shopping list."
        "If the user authorizes it, you can purchase the items using the tool purchase_items."
    ),
    tools=[get_shopping_cart, add_to_shopping_cart, get_budget, search_for_item, purchase_items],
)

profile = UserProfile(id="123", name="Alex", shopping_cart=[])
print("You are now chatting with the shopping assistant. Type 'exit' to end the conversation.")
convo_items: list[TResponseInputItem] = []
while True:
    user_input = input("You: ")

    if user_input == "exit":
        print("Goodbye!")
        break

    convo_items.append({"content": user_input, "role": "user"})
    result = await Runner.run(shopping_agent, convo_items, context=profile)
    
    print(f"Shopping Assistant: {result.final_output}")
    
    convo_items = result.to_input_list()

You are now chatting with the shopping assistant. Type 'exit' to end the conversation.
Shopping Assistant: Hi there! How can I assist you with your grocery shopping today?
Shopping Assistant: I'm sorry to hear that you have a complaint. While I primarily assist with shopping tasks, I'd be glad to help in any way I can. Could you please provide more details about your complaint?
Goodbye!


In [6]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

class UserProfile(BaseModel):
    id: str
    name: str
    admin: bool

class HomeworkCheatDetectionOutput(BaseModel):
    attempting_cheat: bool
    explanation: str

homework_cheat_guardrail_agent = Agent(
    name="Homework Cheat Detector",
    instructions=(
        "Determine if the user's query resembles a typical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable. "
        " Cheating: 'Fill in the blank: The capital of France is ____.',"
        " 'Which of the following best describes photosynthesis? A) Cellular respiration B) Conversion of light energy C) Evaporation D) Fermentation.'"
        " Not-Cheating: 'What is the capital of France?', 'Explain photosynthesis.'"
    ),
    output_type=HomeworkCheatDetectionOutput,
    model="gpt-4o-mini" # usually the guardrail agent can be cheaper than the main agent
)

@input_guardrail
async def cheat_detection_guardrail(
    ctx: RunContextWrapper[UserProfile], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    # Skip guardrail check if user is admin
    if ctx.context.admin:
        return GuardrailFunctionOutput(
            output_info=HomeworkCheatDetectionOutput(attempting_cheat=False, explanation="Admin bypass"),
            tripwire_triggered=False
        )

    detection_result = await Runner.run(homework_cheat_guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=detection_result.final_output,
        tripwire_triggered=detection_result.final_output.attempting_cheat,
    )

study_helper_agent = Agent[UserProfile](
    name="Study Helper Agent",
    instructions="You assist users in studying by explaining concepts or providing guidance, without directly solving homework or test questions.",
    input_guardrails=[cheat_detection_guardrail],
    model="gpt-4o"
)

In [8]:
# This should NOT trigger the cheat detection guardrail since the user is an admin
try:
    response = await Runner.run(study_helper_agent, 
                                "Fill in the blank: The capital of France is ____.", 
                                context=UserProfile(id="123", name="Alex", admin=False))
    print("Guardrail didn't trigger")
    print("Response: ", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details:", str(e))

Homework cheat guardrail triggered
Exception details: Guardrail InputGuardrail triggered tripwire
